<a href="https://colab.research.google.com/github/d-tomas/text-mining/blob/main/notebooks/lecture_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Day 3**: Supervised Learning

## Initial setup

In [ ]:
# Import the required libraries

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer  # Extract TF-IDF weighting schema
from sklearn.feature_extraction.text import CountVectorizer  # Extract TF weighting schema
from sklearn.metrics import accuracy_score  # Calculate the accuracy of the classifier
from sklearn.metrics import confusion_matrix  # Get the confusion matrix
from sklearn.model_selection import cross_val_score  # Cross-validation evaluation
from sklearn.model_selection import train_test_split  # Split the dataset into train and test
from sklearn.naive_bayes import MultinomialNB  # Naïve Bayes algorithm
from sklearn.neighbors import KNeighborsClassifier  # k-NN algorithm
from sklearn.neural_network import MLPClassifier  # Neural Networks algorithm
from sklearn.svm import SVC  # Support Vector Machines algorithm
from sklearn.tree import DecisionTreeClassifier  # Decission tree algorithm
import spacy

# Install the SpaCy model for English texts
spacy.cli.download('en_core_web_sm')

# Load the model (disable some functionalities not used in these exercises to save processing time)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'entity_likner', 'entity_ruler'])

# Download RePEC corpus
!wget https://raw.githubusercontent.com/d-tomas/text-mining/main/datasets/repec_s.csv
# Download cell phone opinions corpus
!wget https://raw.githubusercontent.com/d-tomas/text-mining/main/datasets/cell_phones.csv

## Example 1: text classification

In [ ]:
# Show the first lines of the 'repec_s.csv' file

!head repec_s.csv

In [ ]:
# Loading data from file into a Pandas DataFrame

data = pd.read_csv('repec_s.csv')
data

In [ ]:
# Keep the contents of the abstract for classification

corpus = data['abstract']  # Store the abstracts
y = data['jel']  # Store the JEL category

In [ ]:
# Plot the number of instances in each class

plt.figure(figsize=(10,8))
sns.countplot(x=y)
plt.show()

# F:	International Economics
# I:	Health, Education, and Welfare
# R:	Urban, Rural, Regional, Real Estate, and Transportation Economics
# M:	Business Administration and Business Economics | Marketing | Accounting | Personnel Economics

In [ ]:
# Preprocessing
# For each abstract: remove punctuation, remove stopwords, and lowercase

def normalise(text):
  document = nlp(text)  # Process the text with SpaCy
  document = [token for token in document if not token.is_punct]  # Remove punctuation
  document = [token for token in document if not token.is_stop]  # Remove stopwords
  document = [token.lower_ for token in document]  # Lowercase
  return ' '.join(document)

corpus_normalised = corpus.map(normalise)
corpus_normalised

In [ ]:
# Create a 'classify' function that performs the training and testing
# Input parameters:
# - corpus: the dataset containing the text for train and test
# - model_name: the name of the algorithm that we want to use ('DT', 'KNN', 'MLP', 'NB' or 'SVM')
# - evaluation_type: the type of evaluation, train/test split ('split') or cross-validation ('cv)
# The function returns the trained model and the vectorizer
# Both are required if we want to perform predictions in the future based on this model

def classify(corpus, model_name, evaluation_type):
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(corpus)

  if model_name == 'DT':
    model = DecisionTreeClassifier()  # Decission tree
  elif model_name == 'KNN':
    model = KNeighborsClassifier()  # k-NN
  elif model_name == 'MLP':  
    model = MLPClassifier()  # Neural network
  elif model_name == 'NB':
    model = MultinomialNB()  # Naïve Bayes
  else:
    model = SVC(kernel = 'linear')  # SVM

  # The user chooses to evaluate with train/test split
  if evaluation_type == 'split':
    # Split into training (80%) and test (20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Prediction on the test set
    predictions = model.predict(X_test)

    # Calculate the accuracy of the algorithm
    print('Accuracy: {:.2%}\n'.format(accuracy_score(predictions, y_test)))
    print('Confusion matrix:')

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, predictions), annot=True, linewidth=3)
    plt.yticks(rotation=0)
    plt.show()
  # The user chooses to evaluate with k-fold cross validation
  elif evaluation_type == 'cv':
    scores = cross_val_score(model, X, y, cv=5)  # 5-fold evaluation
    print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  else:
    print('Unknown evaluation type')
  
  return model, vectorizer

In [ ]:
# Possible values for model name: DT, KNN, MLP, NB, SVM
# Possible values for evaluation type: split, cv
# Returns the trained model and the vectorizer for further predictions

model_repec, vectorizer_repec = classify(corpus, 'SVM', 'split')

In [ ]:
# Prediction for a new (never seen before) sample

new_input = ['The prize of bananas and its correlation with global warming']

new_input = vectorizer_repec.transform(new_input)  # Transform the new instance following the same procedure used when the model was created
label = model_repec.predict(new_input)  # Predict the label for the new instance (F, I, R or M)

if label == 'F':
  print('International Economics')
elif label == 'I':
  print('Health, Education, and Welfare')
elif label == 'R':
  print('Urban, Rural, Regional, Real Estate, and Transportation Economics')
elif label == 'M':
  print('Business Administration and Business Economics | Marketing | Accounting | Personnel Economics')
else:
  print('Unknown class')

### Exercise

In [ ]:
# Test with titles instead of abstracts
# Tip: corpus = data['title']

# Try using different n-gram sizes
# Tip: vectorizer = TfidfVectorizer(ngram_range=(1,2))  # Uses 1-grams and 2-grams

# Try using TF weighting schema
# Tip: vectorizer = CountVectorizer()

## Example 2: sentiment analysis

In [ ]:
# Check the first lines of the file containing cell phones opinions

!head cell_phones.csv

In [ ]:
# Loading data from file

data = pd.read_csv('cell_phones.csv')
data

In [ ]:
# Extract the comments and labels

corpus = data['content']  # Store the comments
y = data['opinion']  # Store positive or negative labels

In [ ]:
# Plot the classes

plt.figure(figsize=(5,4))
sns.countplot(x=y)
plt.show()

# POS: positive opinion
# NEG: negative opinion

In [ ]:
# Preprocessing
# Re-use the 'normalise' function

corpus_normalised = corpus.map(normalise)
corpus_normalised

In [ ]:
# Use the 'classify' function as before
# Possible values for model name: DT, KNN, MLP, NB, SVM
# Possible values for evaluation type: split, cv

model_phones, vectorizer_phones = classify(corpus, 'SVM', 'split')

In [ ]:
# Prediction

new_input = ['I love this phone!!']
new_input = vectorizer_phones.transform(new_input)
label = model_phones.predict(new_input)  # Predict the label for the new instance (POS o NEG)

if label == 'POS':
  print('Positive opinion')
elif label == 'NEG':
  print('Negative opinion')
else:
  print('Unknown class')

### Exercise

In [ ]:
# Create a wordcloud of positive opinions
# Tip: data = data[data['opinion'] == 'POS']
# Tip: from wordcloud import WordCloud  # Required library

In [ ]:
# Create a wordcloud of negative opinions

# References

* [RePEC](http://www.repec.org/)
* [JEL Classification System](https://www.aeaweb.org/econlit/jelCodes.php?view=jel)
